In [1]:
import numpy as np

# Introduction
Build MiniFlow, which will be our own version of [TensorFlow](http://tensorflow.org/).

Goal od this lab is to demystify two concepts:
* Backpropagation - process by which neural networks update the weights of the network over time.
* Differentiable Graphs - graphs where nodes are differentiable functions. Useful for visual aids for understanding and calculating complicated derivatives.  TensorFlow is a framework for creating differentiable graphs.

With graphs and backpropagation, you will be able to create your own nodes and properly compute the derivatives.

# Graphs
![Example Neural Network](Lesson5/example-neural-network.png)

The graph consists of **nodes** and **edges**.  These nodes and edges are used in mathematical functions such as linear combination and activation functions.

Outputs from previous layers are used as inputs in subsequent layers, where a mathematical function is performed on that input. (Excludes the input layer)  An example is a node representing $f(x, y) = x + y$, where $x$ and $y$ are input values from nodes in the previous layer.

## Forward Propagation
By propagating values from the first layer (the input layer) through all the mathematical functions represented by each node, the network outputs a value.

## Graphs
There are generally two steps to create a neural network:
1. Define the graph of nodes and edges
2. Propagate values through the graph

This is how MiniFlow will work.

# MiniFlow Architecture
Node will be represented using Python class.  Each Node class will have two list.  One to correspond to its input nodes and one for it's output nodes.  Each node will eventually calculate a value that represents its output. Let's initialize the value to None to indicate that it exists but hasn't been set yet.  Each node will need to be able to pass values forward and perform backpropagation.

In [2]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node recieves values
        self.inbound_nodes = inbound_nodes
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        # A calculated value
        self.value = None
        # For each inbound Node here, add this Node as an outbound Node to that Node.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        
        def forward(self):
            """
            Forward propagation.
            
            Compute the output value based on `inbound_nodes` and
            store the result in self.value.
            """
            raise NotImplemented        

## Nodes that Calculate
Node defines the base set of properties that ever node holds, only specialized subclasses of Node will end up in the graph.  We will build subclasses of Node that can perform calculations and hold values.  Unlike the other subclasses of Node, the Input subclass does not actually calculate anything. The Input subclass just holds a value, such as a data feature or a model parameter (weight/bias).

You can set value either explicitly or with the forward() method. This value is then fed through the rest of the neural network.

In [3]:
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator.
        Node.__init__(self)
        
    # NOTE: Input node is the only node where the value
    # may be passed as an argument to forward().
    #
    # All other node implementations should get the value
    # of the previous node from self.inbound_nodes
    #
    # Example:
    # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        # Overwrite the value if one is passed in
        if value is not None:
            self.value = value

## The Add Subclass
Add, which is another subclass of Node, actually can perform a calculation (addition).  Unlike the Input class, which has no inbound nodes, the Add class takes 2 inbound nodes, x and y, and adds the values of those nodes.

In [4]:
class Add(Node):
    def __init__(self, x, y):
        # You could access `x` and `y` in forward with
        # self.inbound_nodes[0] (`x`) and self.inbound_nodes[1] (`y`)
        Node.__init__(self, [x, y])
        
    def forward(self):
        """"
        Set the value of this node (`self.value`) to the sum of its inbound_nodes.
        """
        x_value = self.inbound_nodes[0].value
        y_value = self.inbound_nodes[1].value
        self.value = x_value + y_value

# Forward Propagation
Two methods to help define and run values through the graph:
* topological_sort()
* forward_pass()

In order to define the network, the order of operations for the nodes need to be defined.  Because the input to some nodes depend on the outputs of others, the graph needs to be flattened in such a war where all the input dependencies for each node are resolved before trying to run its calculation. This is a a technique called **topological sort**.

![Topological sort](Lesson5/topological-sort.001.jpeg)

`topological_sort()` implements topological sorting using Kahn's Algorithm.  It returns a sorted list of nodes in which all of the calculations can run in series. Takes in a `feed_dict`, which is how we initially set a value for an Input node and represented by the python dict.  An example use case is:
```python
# Define 2 `Input` nodes.
x, y = Input(), Input()

# Define an `Add` node, the two above`Input` nodes being the input.
add = Add(x, y)

# The value of `x` and `y` will be set to 10 and 20 respectively.
feed_dict = {x: 10, y: 20}

# Sort the nodes with topological sort.
sorted_nodes = topological_sort(feed_dict=feed_dict)
```

### Setup
Quiz Objectives

* Open nn.py below. You don't need to change anything. I just want you to see how MiniFlow works.
* Open miniflow.py. Finish the forward method on the Add class. All that's required to pass this quiz is a correct implementation of forward.

### forward_pass and topological_sort implementations

In [5]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value


### nn.py

In [6]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""

#from miniflow import *

x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


10 + 5 = 15 (according to miniflow)


# Learning and Loss

Like MiniFlow in its current state, NN take inputs and produce outputs.  But MiniFlow can't improve accuracy, while others can.

## The Linear Function

![Linear Node](Lesson5/Linear-Node.png)

A simple artificial neuron depends on three components:
* inputs, $x$ (vector)
* weights, $w$ (vector)
* bias, $b$ (scalar)

The output is the weighted sum of the inputs plus the bias.
$$
o = \sum_ix_iw_i+b
$$

By varying the weights, you can vary the amount of influence any given input has on the output.  Learning aspect of NN takes place during backpropagation

In [7]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.

        Your code goes here!
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

### nn2.py

In [8]:
"""
NOTE: Here we're using an Input node for more than a scalar.
In the case of weights and inputs the value of the Input node is
actually a python list!

In general, there's no restriction on the values that can be passed to an Input node.
"""
#from miniflow import *

inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {
    inputs: [6, 14, 3],
    weights: [0.5, 0.25, 1.4],
    bias: 2
}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output) # should be 12.7 with this example

12.7


# Linear Transform
This section talked about how we will be using linear algebra and matrix multiplication for linear transformations. Now we will have a n x k weight (W) matrix and a 1 x n input (X) matrix.  To find the transformation we will multiply X\*W then add b.  It goes on to explain how instead of processing only one input at a time. we can process multiple.  So the input matrix will be m x n where m is the number data points we will feed the NN at a time.  The calculation is still X\*W+b but b is "broadcasted" over each row.  Then we had to implement this new Linear Transformation.  My code from the previous sevtion worked for this one.  I tweaked it a little to make it more readable. ^_^

# Sigmoid Function
Neural networks take advantage of alternating transforms and activation functions to better categorize outputs. The sigmoid function is among the most common activation functions.
$$
sigmoid(x) = \frac{1}{1+e^{-x}}
$$
![Sigmoid](Lesson5/Sigmoid.png)

Linear transforms are great for simply shifting values, but neural networks often require a more nuanced transform.  Now we will be tackling activation.  That's the idea of a binary output behavior.  One way to accomplish this is by using a step function, but these are no good because they aren't differentiable.  So one solution is to use the Sigmoid function!  Since it only takes in one paramater, we first feed the precious layer into the transformer node from above. Then pass the output to the activation function.

Concuptually, the sigmoid function makes decisions. When given weighted features from some data, it indicates whether or not the features contribute to a classification.

In [9]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used later with `backward` as well.

        `x`: A numpy array-like object.

        Return the result of the sigmoid function.

        Your code here!
        """
        return 1. / (1.+np.exp(-x))

    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function, `_sigmoid`.

        Your code here!
        """
        # This is a dummy value to prevent numpy errors
        # if you test without changing this method.
        self.value = self._sigmoid(self.inbound_nodes[0].value)

### nn3.py

In [10]:
"""
This network feeds the output of a linear transform
to the sigmoid function.

Finish implementing the Sigmoid class in miniflow.py!

Feel free to play around with this network, too!
"""

import numpy as np
#from miniflow import *

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


# Cost
The sigmoid function is separate from forward so that we may use it for backwardpropagation as well.  Now we will move on to accuracy.  There are many techniques for defining the accuracy of a neural network, all of which center on the network's ability to produce values that come as close as possible to known correct values. People use different names for this accuracy measurement often terming it loss or cost. 

We will calculate the cost using the mean squared error (MSE)
$$
C(w, b) = \frac{1}{m}\sum_x(y(x)-a)^2
$$
* $w$: collection of all weights
* $b$: all biases
* $m$: total number of training examples
* $a$: the approximation of $y(x)$ by the network, both vectors of the same length

The collection of weights is all the weight matrices flattened into vectors and concatenated to one big vector. The same goes for the collection of biases except they're already vectors so there's no need to flatten them prior to the concatenation.  Good way to abstract all the weights and biases used in the NN and makes some things easier to write as we'll see soon in the upcoming descent sections.

The cost, $C$, depends on the difference between the correct output, $y(x)$, and the network's output, $a$.

In [11]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        # TODO: your code here
        self.value = sum(np.square(y - a)) / len(y)
        
def forward_pass(graph):
    """
    Performs a forward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

### nn4.py

In [12]:
"""
Test your MSE method with this script!

No changes necessary, but feel free to play
with this script to test your network.
"""

import numpy as np
#from miniflow import *

y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

[ 23.41666667]


# Gradient Descent Part 1
## Backpropagation

Next is to start a backwards pass, which starts with backpropagation. Backpropagation is the process by which the network runs error values backwards.

During the process, the network calculates the way in which the weights need to change (also called the gradient) to reduce the overall error of the network. Changing the weights usually occurs through a technique called gradient descent.

## Gradient Descent
Think of gradient descent as a ball on the slope of a valley.  In the ideal situation the ball rests at the bottom of this valley.  If we start off with random weights then this is like starting the ball on a random spot in the valley.  Our goal is to move it to the minimum.  At each iteration we find the gradient.  The gradient is the slop of the valley at the point where the ball is at. Then we move the ball slightly in that direction.  Eventually we will converge to a minimum.  The only issue is that it is not always the global min.  This concept can be extended to n dimensions.

![Gradient Descent 1](Lesson5/gradient-descent-1.png)
![Gradient Descent 2](Lesson5/gradient-descent-2.png)

# Gradient Descent Part 2
## Journey to the Bottom of the Valley

Gradient provides the direction of steepest ascent.  We put a minus sign in front to get the steepest descent. It gives us a vector of numbers, each number represents the amount by which we should adjust a corresponding weight or bias in the NN.

The learning rate is how much *force* we use to push the ball.  Be careful in using too much force because we may overshoot the minimum and start diverging.

Knowing a good learning rate is more of a guessing game, but values in the range 0.1 to 0.0001 work well.  Range 0.001 to 0.0001 is popular as 0.1 and 0.01 are sometimes too large.

Formula for gradient descent:   **x = x - learning_rate * gradient_of_x**

# Backpropagation
## The Gradient and Backpropagation

> *In order to figure out how we should alter a parameter to minimize the cost,  
> we must first find out what effect that parameter has on the cost.*

We use **backpropagation** or **reverse-mode differentation**.  It's a clever application of the **chain rule**.

## Derivatives
The derivative tells us how something changes with respect to something else.

## Chain Rule
We calculate the derivative of the cost with respect to each parameter in the network.  The gradient is a vector of all these derivatives.

In reality, neural networks are a composition of functions, so computing the derivative of the cost w.r.t a parameter isn't quite as straightforward as calculating the derivative of a polynomial function like $f(x)=x^2$

Say we have a new function $f(g(x))$. We can calculate the derivative of f w.r.t $x$ ($\frac{\partial f}{\partial x}$) by applying the chain rule.

$$
\frac{\partial f}{\partial x} = \frac{\partial g}{\partial x} \frac{\partial f}{\partial g}
$$

> *In order to know the effect **x** has on **f**, we first need to know the effect **x** has on **g**,  
> and then the effect **g** has on **f**.*

A more complex example
```python
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(l2, y)
```

Can be written as a composition of functions 
```
MSE(Linear(Sigmoid(Linear(X, W1, b1)), W2, b2), y)
```
![MiniFlow NN](Lesson5/miniflow-nn-graph.001.jpeg)

Goal is to adjust the weights and biases represented by the Input nodes W1, b1, W2, b2 such that the cost is minimized.

First we unwrap the derivative of **cost** w.r.t. **l1** (the input to the Sigmoid node). Once again, apply the chain rule:
![Deriving](Lesson5/deriving.png)

## New Code
There have been a couple of changes to MiniFlow since we last took it for a spin:

The first being the Node class now has a backward method, as well as a new attribute self.gradients, which is used to store and cache gradients during the backward pass.

### MiniFlow New

In [13]:
"""
Implement the backward method of the Sigmoid node.
"""
import numpy as np


class Node(object):
    """
    Base class for nodes in the network.

    Arguments:

        `inbound_nodes`: A list of nodes with edges into this node.
    """
    def __init__(self, inbound_nodes=[]):
        """
        Node's constructor (runs when the object is instantiated). Sets
        properties that all nodes need.
        """
        # A list of nodes with edges into this node.
        self.inbound_nodes = inbound_nodes
        # The eventual value of this node. Set by running
        # the forward() method.
        self.value = None
        # A list of nodes that this node outputs to.
        self.outbound_nodes = []
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        # Sets this node as an outbound node for all of
        # this node's inputs.
        for node in inbound_nodes:
            node.outbound_nodes.append(self)

    def forward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `forward` method.
        """
        raise NotImplementedError

    def backward(self):
        """
        Every node that uses this class as a base class will
        need to define its own `backward` method.
        """
        raise NotImplementedError


class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object.
        self.gradients = {self: 0}
        # Weights and bias may be inputs, so you need to sum
        # the gradient from output gradients.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] += grad_cost * 1


class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)


class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            """
            TODO: Your code goes here!

            Set the gradients property to the gradients with respect to each input.

            NOTE: See the Linear node and MSE node for examples.
            """
            f_x = self.value
            self.gradients[self.inbound_nodes[0]] += grad_cost * (f_x * (1-f_x))


class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calculates the gradient of the cost.

        This is the final node of the network so outbound nodes
        are not a concern.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted Nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()

def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # TODO: update all the `trainables` with SGD
    # You can access and assign the value of a trainable with `value` attribute.
    for t in trainables:
        t.value = t.value - learning_rate*t.gradients[t]

### nn5.py

In [14]:
"""
Test your network here!

No need to change this code, but feel free to tweak it
to test your network!

Make your changes to backward method of the Sigmoid class in miniflow.py
"""

import numpy as np
#from miniflow import *

X, W, b = Input(), Input(), Input()
y = Input()
f = Linear(X, W, b)
a = Sigmoid(f)
cost = MSE(y, a)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2.], [3.]])
b_ = np.array([-3.])
y_ = np.array([1, 2])

feed_dict = {
    X: X_,
    y: y_,
    W: W_,
    b: b_,
}

graph = topological_sort(feed_dict)
forward_and_backward(graph)
# return the gradients for each Input
gradients = [t.gradients[t] for t in [X, y, W, b]]

"""
Expected output

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]
"""
print(gradients)

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]


# Stochastic Gradient Descent

Stochastic Gradient Descent (SGD) is a version of Gradient Descent where on each forward pass a batch of data is randomly sampled from the total dataset.  Ideally, the entire dataset would be fed into the neural network on each forward pass, but in practice, that's not practical due to memory constraints.  SGD is an approximation of Gradient Descent, the more batches processed by the neural network, the better the approximation.

Naive implementation of SGD:
1. Randomly sample a batch of data from the total dataset.
2. Running the network forward and backward to calculate the gradient (with data from 1)
3. Apply the gradient descent update.
4. Repeat steps 1-3 until convergence or the loop is stopped by another mechanism (i.e. the number of epochs)

Network's loss should generally trend downwards, indicating more useful weights and biases over time.

Gradient descent update equation:
$$
x = x - \alpha \cdot \frac{\partial cost}{\partial x}
$$

ach example in the dataset is a description of a house in the Boston suburbs, the description consists of 13 numerical values (features). Each example also has an associated price. With SGD, we're going to minimize the MSE between the actual price and the price predicted by the neural network based on the features.

**Look at sgd_update abouve**

### nn6.py
**Note: Can't be run**

In [ ]:
"""
Check out the new network architecture and dataset!

Notice that the weights and biases are
generated randomly.

No need to change anything, but feel free to tweak
to test your network, play around with the epochs, batch size, etc!
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample
from miniflow import *

# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 10
# Total number of examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        forward_and_backward(graph)

        # Step 3
        sgd_update(trainables)

        loss += graph[-1].value

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))


# Vector, Matrix, and Tensor Derivatives

Summary of [*Vector, Matrix, and Tensor Derivatives*](http://cs231n.stanford.edu/vecDerivs.pdf) by Erik Learned-Miller

Purpose to to help you learn to take derivatives of Vectors, Matrices, andHigher Order Tensors (arrays with three dimensions or more), and to help you take derivatives *with respect to* vectors, matrices and higher order tensors.

## 1 Simplify, simplify, simplify
Don't do too many things at once.
* Taking derivatives of multiple components simultaneously
* Taking derivatives in the prescence of summation notation
* Applying the chain rule

### 1.1 Expanding notation into explicit sums and equations for each component

It is often useful to write out the explicit formula for *a single scalar element* of the output in terms of nothing but *scalar variables*.